In [1]:
import torch
import bock
from bock import SingleTaskGP, ExpectedImprovement, bo_step, CustomCylindricalGP
from IPython.display import clear_output
def opt_func(X):
    return X.norm(dim=1)

In [2]:
# dimensionality of the problem
n_dim = 5
# number of init measurements
num_init = 20
# size of search space
x_step = 5.
# center of search space
center = 0.7

borders = []
for i in range(n_dim):
    borders.append((center - x_step, center + x_step))
borders = torch.tensor(borders).float()
soboleng = torch.quasirandom.SobolEngine(dimension=n_dim)
X = soboleng.draw(num_init).float()
X = X * (borders[:, 1] - borders[:, 0]) + borders[:, 0]
y = opt_func(X)

In [3]:
X, y

(tensor([[ 0.7000,  0.7000,  0.7000,  0.7000,  0.7000],
         [ 3.2000, -1.8000,  3.2000, -1.8000,  3.2000],
         [-1.8000,  3.2000, -1.8000,  3.2000, -1.8000],
         [-0.5500, -0.5500,  1.9500, -3.0500,  4.4500],
         [ 4.4500,  4.4500, -3.0500,  1.9500, -0.5500],
         [ 1.9500, -3.0500, -0.5500, -0.5500, -3.0500],
         [-3.0500,  1.9500,  4.4500,  4.4500,  1.9500],
         [-2.4250, -1.1750, -1.1750,  2.5750,  1.3250],
         [ 2.5750,  3.8250,  3.8250, -2.4250, -3.6750],
         [ 5.0750, -3.6750,  1.3250,  5.0750, -1.1750],
         [ 0.0750,  1.3250, -3.6750,  0.0750,  3.8250],
         [-1.1750, -2.4250,  5.0750,  1.3250,  0.0750],
         [ 3.8250,  2.5750,  0.0750, -3.6750,  5.0750],
         [ 1.3250,  0.0750, -2.4250,  3.8250,  2.5750],
         [-3.6750,  5.0750,  2.5750, -1.1750, -2.4250],
         [-3.3625,  0.3875,  4.1375, -0.2375, -1.4875],
         [ 1.6375,  5.3875, -0.8625,  4.7625,  3.5125],
         [ 4.1375, -2.1125, -3.3625, -2.7375,  1

In [4]:
state_dict = None
for i in range(100):
    GP = lambda X, y, noise, borders: CustomCylindricalGP(X, y.view(-1, 1), noise, borders)
    acquisition = lambda gp, y: ExpectedImprovement(gp, y.min(), maximize=False)
    objective = lambda x: opt_func(x)
    X, y, gp = bo_step(
        X,
        y.view(-1, 1),
        noise=torch.zeros_like(y), # placeholder, not used
        objective=objective,
        bounds=borders.t(),
        GP=GP,
        acquisition=acquisition,
        q=1,
        state_dict=state_dict
    )
    state_dict = gp.state_dict()
    clear_output()
    print("Last measurement", X[-1], y[-1])
    print("Best measurement", X[y.argmin()], y.min())

Last measurement tensor([ 0.1522,  0.3497,  0.2695,  0.2673, -0.4916]) tensor([0.7288])
Best measurement tensor([0.0120, 0.1130, 0.1032, 0.1884, 0.1907]) tensor(0.3089)
